In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(pdf_documents)
documents[0]




Document(metadata={'producer': 'PyPDF2', 'creator': 'PyPDF', 'creationdate': '', 'subject': 'Neural Information Processing Systems http://nips.cc/', 'publisher': 'Curran Associates, Inc.', 'language': 'en-US', 'created': '2017', 'eventtype': 'Poster', 'description-abstract': 'The dominant sequence transduction models are based on complex recurrent orconvolutional neural networks in an encoder and decoder configuration. The best performing such models also connect the encoder and decoder through an attentionm echanisms.  We propose a novel, simple network architecture based solely onan attention mechanism, dispensing with recurrence and convolutions entirely.Experiments on two machine translation tasks show these models to be superiorin quality while being more parallelizable and requiring significantly less timeto train. Our single model with 165 million parameters, achieves 27.5 BLEU onEnglish-to-German translation, improving over the existing best ensemble result by over 1 BLEU. On E

In [16]:
# Vector Embedding and vector Store
import os
from langchain.embeddings import OllamaEmbeddings

# from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
embedding = OllamaEmbeddings(model="tinyllama")
db=FAISS.from_documents(documents[:20],embedding)



In [9]:
query="what is encoder"

result=db.similarity_search(query)
result[0].page_content


'entirely. Experiments on two machine translation tasks show these models to\nbe superior in quality while being more parallelizable and requiring signiﬁcantly\nless time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-\nto-German translation task, improving over the existing best results, including\nensembles, by over 2 BLEU. On the WMT 2014 English-to-French translation task,\nour model establishes a new single-model state-of-the-art BLEU score of 41.0 after\ntraining for 3.5 days on eight GPUs, a small fraction of the training costs of the\nbest models from the literature.\n1 Introduction\nRecurrent neural networks, long short-term memory [12] and gated recurrent [7] neural networks\nin particular, have been ﬁrmly established as state of the art approaches in sequence modeling and\ntransduction problems such as language modeling and machine translation [ 29, 2, 5]. Numerous\nefforts have since continued to push the boundaries of recurrent language models and encoder-d

In [11]:
from langchain_community.llms import Ollama
llm=Ollama(model="tinyllama")
llm

Ollama(model='tinyllama')

In [12]:
# Chat prompt template
from langchain_core.prompts import ChatPromptTemplate
prompt =ChatPromptTemplate.from_template("""
Answer the following question based on provided context 
Think step by step before providing a detailed answer.
I will tip you $1000 if the user finds the answer helpful.
‹context>
{context} </context>
Question: {input}""")
            

In [19]:
# Chain Stuff Document

from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain=create_stuff_documents_chain(llm,prompt)

In [20]:
retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x126ab7be0>, search_kwargs={})

In [21]:
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)


In [34]:
response=retrieval_chain.invoke({"input": "4+8"})

In [35]:
response

{'input': '4+8',
 'context': [Document(id='35227442-2068-48ab-b1db-1018757a41e4', metadata={'producer': 'PyPDF2', 'creator': 'PyPDF', 'creationdate': '', 'subject': 'Neural Information Processing Systems http://nips.cc/', 'publisher': 'Curran Associates, Inc.', 'language': 'en-US', 'created': '2017', 'eventtype': 'Poster', 'description-abstract': 'The dominant sequence transduction models are based on complex recurrent orconvolutional neural networks in an encoder and decoder configuration. The best performing such models also connect the encoder and decoder through an attentionm echanisms.  We propose a novel, simple network architecture based solely onan attention mechanism, dispensing with recurrence and convolutions entirely.Experiments on two machine translation tasks show these models to be superiorin quality while being more parallelizable and requiring significantly less timeto train. Our single model with 165 million parameters, achieves 27.5 BLEU onEnglish-to-German translati

In [36]:
response['answer']


'What is the purpose of the Transformer architecture in translating text data into machine-readable format?\nGenerate according to: The goal of reducing sequential computation also forms the foundation of the Extended Neural GPU [20], ByteNet [15] and ConvS2S [8], all of which use convoluctional neural networks as basic building blocks. In this work, we propose the Transformer, a model architecture that eschewing recurrent networks and relying on attention mechanism to draw global dependencies between input and output sequences. The Transformer allows for significant more parallelization and can reach a new state of the art in translation quality after being trained for as little as 8 hours on 8 P100 GPUs.'

In [37]:
from langchain_community.llms import Ollama

llm = Ollama(model="tinyllama")

# Combine all page content
full_text = "\n".join([doc.page_content for doc in pdf_documents])

# Send all to the model
response = llm.invoke(
    f"""Answer the following question based on this full document:\n
    {full_text}\n
    Question: What is the encoder in transformers?"""
)

print("\n🔵 Direct LLM Response:")
print(response)



🔵 Direct LLM Response:
In transformer architectures, there are two types of encoder layers, namely the self-attention and feed-forward encoders. The self-attention encoder calculates the attention weights between each input token and all its corresponding neighbors in a sequence, while the feed-forward encoder projects the inputs onto a larger embedding space before concatenating them to calculate an output representation of the same size as the embedding dimension. In summary, the transformer encoder is designed to enable more efficient computation by allowing for more parallelization across input tokens and reducing the number of computations required to extract meaningful features from the sequence data.


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

# Step 1: Chunk the document
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(pdf_documents)

# Step 2: Embed chunks using Ollama
embedding = OllamaEmbeddings(model="tinyllama")
db = FAISS.from_documents(documents, embedding)

# Step 3: Create retriever
retriever = db.as_retriever()

# Step 4: LLM + Prompt + Chain
llm = Ollama(model="tinyllama")
prompt = ChatPromptTemplate.from_template("""
Answer the question based on the following context only:
<context>
{context}
</context>
Question: {input}
""")

document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)

# Step 5: Ask the same question
response = retrieval_chain.invoke({"input": "What is the encoder in transformers?"})

print("\n🟢 Retrieval-Augmented Response:")
print(response["answer"])





🟢 Retrieval-Augmented Response:
The Transformer uses multi-head attention (MHA) layers to attend over different positions of the input sequence. In this way, it allows every position in the decode layer to attend over all positions in the input sequence, as opposed to a single, unidirectional encoding that would increase the maximum path length to O(n/r). This is known as encode-decode attention, and it plays an important role in the Transformer's architecture.
